In [228]:
import math
from si_prefix import si_format



def to_e_series(value, series=-1, suffix="", si=True, mode="up"):
        val = 0
        prev_val = 0
        
        
        exponent = int(math.log10(value))
        mantissa = value / math.pow(10, exponent)
        
        while(mantissa < 1):
            mantissa *= 10
            exponent -= 1
            
        
        if (series > 0) and (mode == "up"):
            
            for x in range(0, series):
                
                val = round((10**x)**(1/series), 2) 
                
                if val >= mantissa:     
                    result = val*10**exponent
                    if (si == False):    
                        return result
                    else: 
                        return (f"{si_format(result, precision=2)}{suffix}")
               
            result = 10*10**exponent
            if (si == False):    
                    return result
            else:
                    return(f"{si_format(result, precision=2)}{suffix}")
        
        elif (series > 0) and (mode == "closest"):
            
            for x in range(0, series):
                prev_val = val
                val = round((10**x)**(1/series), 2)      

                if val >= mantissa:    
                    
                    if (abs(mantissa-val) > abs(mantissa-prev_val)): #calc lower difference
                        val = prev_val
                        
                    result = val*10**exponent
                    if (si == False):    
                        return result
                    else: 
                        return(f"{si_format(result, precision=2)}{suffix}")

            
            if (abs(mantissa-9.53) < abs(mantissa -10)): 
                result = 9.53*10**exponent
            else:                
                result = 10*10**exponent
            
            if (si == False):    
                return result
            else: 
                return si_format(result, precision=2)
        elif (series == -1):
            return(f"{si_format(value, precision=2)}{suffix}") 
        
        

In [334]:
# Design Generator for LM5085 - see Datasheet https://www.ti.com/lit/ds/symlink/lm5085.pdf




#-----------------------------
# Parameters: 

v_in_min = 24        # Minimum input voltage (absolute min: 4.5V) [V]
v_in_max = 48        # Maximum input voltage (absolute max: 75V) [V]

v_out = 12           # Output Voltage [V]

v_or = 0.01          # Output Ripple [V] !!! MUST BE AT LEAST 25 mV AT FB
ripple_factor = 0.4 # suggestion of 0.2-0.4 of i_out_max if the minimum output current is 0


i_out_max = 2        # Maximum output current [A]
i_out_min = i_out_max * ripple_factor # Minimum output load

i_or_max = 2*i_out_min

i_cl_min = i_out_max + i_or_max



f_sw = 500e3    # Switching frequency (abs. max. 1MHz) [Hz]

# Output Resistor Selection; Select R2 from 1..20kΩ

R2 = 10000;         # [Ω]
R1 = R2 / ((v_out/1.25)-1)




# Rt - select PFET first
# SI3127DV-T1-GE3CT - https://www.vishay.com/docs/64282/si3127dv.pdf

# See turn-off and turn-on delay 

t_on_min = 8e-9       # [ns]
t_on_max = 16e-9      # [ns]

t_off_min = 35e-9    # [ns]
t_off_max = 53e-9      # [ns]

t_d_max = t_off_max - t_on_min
t_d_min = t_off_min - t_off_max

t_d = (t_d_max + t_d_min / 2) + (50e-9)


# select R_t to determine switching freq




rt = (((v_out * (v_in_min - 1.56)) / (1.45e-7 * v_in_min * f_sw)) - (t_d * (v_in_min - 1.56) / (1.45e-7 ))- 1.4) *1000




# minimum turn on time, no less than 150ns



# set current limit through R_adj


r_sense = 0.01 # [Ω]

r_adj = (r_sense * i_cl) / (40e-6)


#current rise during on-time
i_or_max = 2 * iout_min


i_cl_min = i_out_max + (i_or_max/2)


L1 = (t_on_max * (v_in_max - v_out)) / i_or_max


# OUTPUT FORMAT


print("Resulting Values for LM5085")
print("-------------------------------------------------------------------------")
print(f"\nInput Parameters:")
print(f"    Input Voltage:          {v_in_min} - {v_in_max} V")
print(f"    Output Voltage:         {v_out} V")
print(f"    Output current:         {to_e_series(i_out_max, suffix='A')}")
print(f"    Switching frequency:    {to_e_series(f_sw, suffix='Hz')}")
print("-------------------------------------------------------------------------")
print("Component Values:")
print("    Feedback Divider: ")

r_fb1_act = to_e_series(R1, 48, si=False, mode="closest")
r_fb2_act = to_e_series(R2, 48, si=False)


print(f"        R_FB1:         {to_e_series(R1, 48, mode='closest', suffix='Ω')}")
print(f"        R_FB2:         {to_e_series(R2, 48, 'Ω')}")

v_out_act = 1.25 * ((r_fb2_act + r_fb1_act) / r_fb1_act)

print(f"        Actual v_out:   {to_e_series(v_out_act, suffix='V')}")
## TODO: don't necessarsily round up, custom option of "smallest deviation"

r_t_act = to_e_series(rt, 48, si=False, mode="closest")


f_sw_act = ( v_out_act * (v_in_min - 1.56 + (r_t_act/1000)/3167)) / (v_in_min * ((1.45e-7 * (r_t_act/1000 + 1.4)) + (t_d * (v_in_min - 1.56 + r_t_act/1000/3167))))
print("\n    Frequency Select:")
print(f"        R_Rt:          {to_e_series(rt, 48, 'Ω')}")

print(f"        Actual Freq:   {to_e_series(f_sw_act, suffix='Hz')}")

print("\n    Current Limiting:")
print(f"        Rsense:        {to_e_series(r_sense, suffix='Ω')}")


r_adj = ((i_cl_min * r_sense) + 0.009) / 32e-6       # voltage drop across r_sense + 9mV current limit comparator offset

r_adj_act = to_e_series(r_adj, 48, si=False, mode="up")


print(f"        R_adj:         {to_e_series(r_adj, si=True, mode='up')}\n")

i_lim_nom = 40e-6 * r_adj_act / r_sense
i_lim_min = ((r_adj_act * 32e-6) - 9e-3) / 0.01
i_lim_max = ((r_adj_act * 48e-6) + 9e-3) / 0.01


print(f"        I_Limit_Min:       {to_e_series(i_lim_min, suffix = 'A')}")
print(f"        I_Limit_Nominal:   {to_e_series(i_lim_nom, suffix = 'A')}")
print(f"        I_Limit_Max:       {to_e_series(i_lim_max, suffix = 'A')}")


c_out = i_or_max / (8* f_sw * v_or)
print(f"\n    Input/Output Capacitors:")
print(f"        C_out              {to_e_series(c_out, 48, suffix= 'F')}")


t_on_max = (1.45e-7* ((r_t_act/1000) + 1.4)) / (v_in_min - 1.56 + ((r_t_act/1000) / 3167)) + (50e-9)
t_on_min = (1.45e-7* ((r_t_act/1000) + 1.4)) / (v_in_max - 1.56 + ((r_t_act/1000) / 3167)) + (50e-9)

c_in = (i_out_max * t_on_max) / 0.1

print(f"        C_in + C_byp       {to_e_series(c_in, 48, suffix= 'F')}")



L1 = (t_on_min * (v_in_max - v_out)) / i_or_max

L1_act = to_e_series(L1, series=48, si=False)

print(f"\n    Inductor L1:     {to_e_series(L1, suffix='H')}")



if v_or < 0.025:
    print("-------------------------------------------------------------------------")
    print(f"Special Config due to Output Ripple < 25mV: [actual: {to_e_series(v_or, suffix='V')}]\n")
    
   
    t_on_max = (1.45e-7* ((r_t_act/1000) + 1.4)) / (v_in_min - 1.56 + ((r_t_act/1000) / 3167)) + (50e-9)
    print(f"    PFET max. On-Time  {to_e_series(t_on_max, suffix ='s')} <--- IMPORTANT: Must be bigger than 150ns, otherwise adjust F_SW")

    v_a = v_out - (0.65*(1-(v_out/v_in_min)))
    #print(v_a)
    
    r3_c1 = ((v_in_min - v_a) * t_on_max) / 0.05
    print(f"    RC Constant: {to_e_series(r3_c1, series=-1, suffix='s')}")
    
    c1 = 2000e-12
    c1_act = to_e_series(value=c1, series=48, si=False)
    print(f"        C1:       {to_e_series(c1, 48, suffix='F')}")
    
    r3 = r3_c1 / c1_act
    r3_act = to_e_series(value=r3, series=48, si=False)
    print(f"        R3:       {to_e_series(r3, 48, suffix='Ω')}")
    

Resulting Values for LM5085
-------------------------------------------------------------------------

Input Parameters:
    Input Voltage:          24 - 48 V
    Output Voltage:         12 V
    Output current:         2.00 A
    Switching frequency:    500.00 kHz
-------------------------------------------------------------------------
Component Values:
    Feedback Divider: 
        R_FB1:         1.15 kΩ
        R_FB2:         10.00 kΩ
        Actual v_out:   12.12 V

    Frequency Select:
        R_Rt:          147.00 kΩ
        Actual Freq:   506.05 kHz

    Current Limiting:
        Rsense:        10.00 mΩ
        R_adj:         1.16 k

        I_Limit_Min:       2.97 A
        I_Limit_Nominal:   4.84 A
        I_Limit_Max:       6.71 A

    Input/Output Capacitors:
        C_out              40.20 µF
        C_in + C_byp       19.60 µF

    Inductor L1:     11.05 µH
-------------------------------------------------------------------------
Special Config due to Output Ripple < 2